# Web Scraping: lo básico


![](https://drive.google.com/uc?id=1cmCRKtVedlfkXlQHKk3CUzk-1MLlMAuu)

# Web Scraping Wikipedia

BeautifulSoup es una librería muy útil para hacer Web Scraping. 

La manera de utilizarla es bastante simple:

En primer lugar, necesitamos importar el **módulo** de la siguiente manera:

In [ ]:
from bs4 import BeautifulSoup

Sin embargo, Python no es un navegador Web, así que no podemos "llamar" directamente la página sin un módulo que se comunique con el servidor. 

Para nuestro ejercicio utilizaremos `urllib.request`, un módulo relativamente simple para comunicarse con páginas web con el protocolo HTTP.

In [ ]:
import urllib.request

Ahora, vamos a guardar en una **variable** la dirección URL que queremos recuperar:

In [ ]:
host = "https://es.wikipedia.org/wiki/Colombia"

El paso siguiente consiste en "preparar la sopa". Para ello utilizaremos dos funciones: una para recuperar la página Web y otra para leer el código HTML. 

Guardaremos cada función en variables, la primera la llamaremos `salsa` y a la segunda `sopa`.

*Observa como utilizamos el módulo `urllib.request` al cual le pasamos una **función** (`.urlopen`) e incluimos un **parámetro** dentro del paréntesis, el cual es la variable que contiene nuestra URL*

In [ ]:
salsa = urllib.request.urlopen(host).read() 
sopa =  BeautifulSoup(salsa, 'html.parser')

print(sopa)
  

El problema, como podrán observar, es que obtenemos todo el código HTML de la página, lo cual es igual a simplemente guardar el sitio. 

Lo que queremos es poder escoger partes específicas de la página para luego guardarlas o aplicarles algún análisis. 

En ese caso, necesitamos identificar en qué **etiquetas** se guarda la información. Empezaremos con la más sencilla, el texto.

La manera más recomendable para hacer este ejercicio consiste en utilizar la herramienta inspeccionar, disponibles en Google Chrome y Mozilla.

Así, escogemos el contenido que queremos recuperar y con el click derecho seleccionamos 'Inspeccionar' (en Chrome) o 'Inspeccionar elemento' (en Mozilla)

![](https://drive.google.com/uc?id=1tc9TZcG4nZo5VdPF_OXvYrgX-lFRHGdM)


Si observas detenidamente verás que alrededor de la palabra 'Colombia' hay dos **etiquetas**. Así se llaman en HTML y otros lenguajes de marcado de texto a los elementos que indican una función que debe cumplir el texto al momento de ser desplegado en el navegador, asimismo, sirven para señalar información propia del sitio Web como el título, la codificación, los metadatos para los motores de búsqueda, los recursos externos como JavaScript o CSS, entre otros.

Se identifican de manera sencilla porque se incluyen entre corchetes angulares así: `<etiqueta></etiqueta>`

La estructura del marcado se hace de manera 'anidada', así una palabra en negrilla está anidada en una etiqueta superior, que sería texto. Para nuestro ejemplo, la estructura iría de la siguiente manera


In [ ]:
%%html
<p>
  <b>Colombia</b>, oficialmente <b>República de Colombia</b>...
</p>

Así, sabemos que la etiqueta para el texto es `<p>` ('párrafo') y otras como `<b>` ('negrilla' o 'bold'), `<a>` ('enlace') o `<i>` ('itálica' o 'cursiva') están anidadas a `<p>`. 

In [ ]:
%%html

<p>
  Este es un texto de párrafo, que puedo incluir una <b>negrilla</b>, un <a href="https://www.google.com/" target ="_blank">enlace</a>, o simplemente un texto en <i>cursiva</i>.
</p>

<b>Asimismo, puedo <i>anidar</i> etiquetas que no están en <p>párrafo</p>. Pero su comportamiento es completamente diferente.</b>

Por otra parte, existen otras etiquetas que indican divisiones más grandes y su etiqueta, como era de suponerse, es `<div>`. Si pasas el cursor sobre los diferentes elementos del panel de inspección, verás como se resaltan esas divisiones y se indica el nombre con el cual se identifican.

![](https://drive.google.com/uc?id=1YXN4MqF_yZC4LA-FKg6XtTtqJMkS51zG)

Así, la etiqueta que alberga el contenido principal del artículo tiene esta forma:

`<div id="bodyContent" class="mw-body-content"></div>`

Donde `id` es el **atributo** que identifica a la etiqueta, es único y no se repite en todo el documento. El atributo `class` remite a una clase en una hoja de estilos CSS. 

De manera simple funciona de la siguiente manera:

In [ ]:
%%html

<!-- A esto se le denomina hoja de estilos
Con este código vamos a indicarle a la página
 que sólo los párrafos que estén en el 'div'
tengan fondo amarillo.
-->

<style>
  .mw-body-content p { background-color: yellow; }
  .mw-body-content h3 { color: red; }
</style>

<!-- Este es el cuerpo del documento -->

<h3>Este es un título normal</h3>
<p>Este es un párrafo de primer nivel</p>

<div id="bodyContent" class="mw-body-content">
  <h3>Este título lleva otro color</h3>
  <p>Este párrafo está dentro de un <code>div</code></p>
</div>

> Obviamente cubrimos una mínima parte del etiquetado HTML y sus atributos. Por eso es recomendable profundizar en el lenguaje para comprender mejor cómo se estructura un sitio Web.

> Un buen lugar para empezar es https://www.w3schools.com/html/. En español, un recurso interesante es https://www.w3.org/Style/Examples/011/firstcss.es.html.

> Para practicar es recomendable utilizar https://codepen.io/





# Web Scraping a una página

Empecemos con un ejercicio simple. Vamos a ejecutar el siguiente código y veamos cuál es el resultado (asegúrate de haber ejecutado las tres primeras celdas de código para que funcione este script)

In [ ]:
salsa = urllib.request.urlopen(host).read()
sopa =  BeautifulSoup(salsa, 'html.parser')

for texto in sopa.find_all('p'):
	print(texto.text)

Expliquemos primero lo que hace este script:



1.   Repetimos el paso de la celda [4] y guardamos en variables la `salsa` y la `sopa`
2.   Usamos un **loop** para seleccionar todos los elementos de párrafo `('p')` en la `sopa`
3.   Le pedimos a la consola que nos muestre el resultado **pero** solamente el texto (no el código HTML), por lo que pasamos la función `.text` para que nos recupere sólo el texto.

El problema es que no nos muestra los encabezados, que se encuentran etiquetados como `<h1>`, `<h2>` y `<h3>`. Si pasamos el código para que nos recupere esos títulos obtenemos lo siguiente:



In [ ]:
# No necesitamos la `sopa` porque ya la tenemos hecha en la celda anterior.

for texto in sopa.find_all(['h1','h2','h3']): # Reemplazamos la etiqueta de párrafo por la de título
	print(texto.text)

Nota que incluimos una nueva forma de pasar nuestra información en la función `find_all()`. En este caso utilizamos una **lista**, que se indica mediante corchetes y cada elemento de la lista se separa por comas `['1', '2', '3'... 'n']`. 

Así, no es necesario escribir un loop para cada encabezado sino podemos hacerlo para todos al mismo tiempo.

Sin embargo, lo que queremos es el texto, tanto de los párrafos como de los títulos. Para ello utilizaremos el siguiente código:

In [ ]:
for titulo in sopa.find_all(['h1','h2','h3']):
	print(titulo.text)
	for parrafo in titulo.next_siblings:
		if parrafo.name and parrafo.name.startswith('h'):
			break
		if parrafo.name == 'p':
			print(parrafo.text)

Con este script recuperamos la información general del artículo incluyendo sus títulos y párrafos. Lo explico para que sepas cómo funciona:



1.   Pasamos por un loop que nos recupere todos los elementos que contengan las etiquetas 'h1', 'h2' o 'h3'.
2.   Le pedimos a python que nos imprima el texto de los encabezados.
3.   Ahora, a cada encabezado le pedimos que nos identifique sus elementos 'hermanos'.
4.   Tenemos ahora que pasar una condición lógica que diga: "Si el elemento hermano es igual a 'p' entonces imprime el párrafo de texto"" (`if parrafo.name == 'p': print(parrafo.text)`). Pero antes debemos indicarle a Python que "Si hay un elemento hermano y este elemento hermano empieza con 'h' entonces no haga nada". Si no pasamos esta condición de esa manera solamente nos escribirá los párrafos después del primer título y no los demás títulos.



# Web Scraping a múltiples páginas

Obviamente, reducir el ejercicio a una sola página no tiene mucho sentido. No es necesario hacer un programa si podemos simplemente guardar la página o copiar el texto de una página Web.

Por eso, el uso más extendido del Web Scraping consiste en recuperar la información de varias páginas con un solo script. 

Vamos a proponernos dos ejercicios:

1.   Recuperar el texto de todas las páginas de una categoría de Wikipedia.
2.   Recuperar el texto de un artículo en todos los idiomas que tenga disponible.

Vamos a utilizar la categoría [Países del mar Caribe](https://es.wikipedia.org/wiki/Categor%C3%ADa:Pa%C3%ADses_del_mar_Caribe) y el artículo [Colombia](https://es.wikipedia.org/wiki/Colombia) para estos ejercicios.

## Recuperar la información de una serie de páginas en una categoría

Primero, vamos a aprovechar lo que ya conocemos y convertiremos el loop para extraer el texto en una **función**. Las funciones son muy útiles porque nos evitan tener que escribir el mismo código varias veces, además, en caso de error es más sencillo arreglar una sola función que buscar por todo el código cada vez que se repita.

En Python las funciones se escriben así:

```python
def mi_funcion(parametros):
        acciones...
```

De esta manera, nuestra función quedará así:

In [ ]:
def ws_texto(miurl):
	salsa2 = urllib.request.urlopen(miurl).read()
	sopa2 =  BeautifulSoup(salsa2, 'html.parser')
	for titulo in sopa2.find_all(['h1','h2','h3']):
		print(titulo.text)
		for parrafo in titulo.next_siblings:
			if parrafo.name and parrafo.name.startswith('h'):
				break
			if parrafo.name == 'p':
				print(parrafo.text)

Lo que hace nuestra función es que con cada URL (`miurl`) prepara una nueva `salsa2` y una nueva `sopa2` para extraer de cada enlace los títulos y el texto. 

Ya que hemos construido nuestra función, pasamos a encontrar los enlaces en la categoría. Como vemos, la página [Países del mar Caribe](https://es.wikipedia.org/wiki/Categor%C3%ADa:Pa%C3%ADses_del_mar_Caribe) está dividida en tres celdas: la primera contiene un mapa y el enlace a Wikimedia Commons, la segunda es un listado de subcategorías, la tercera contiene las páginas en la categoría «Países del mar Caribe». 

Concentraremos la búsqueda en esta última. Para ello, volvemos al inspector y allí podemos identificar que el listado de las páginas se encuentra en el div con el id `mw-pages`. 

![](https://drive.google.com/uc?id=1KJo9nLULPGjhrUVMMFoEo0UuRfqofxIv)

Como requerimos buscar solamente los enlaces, aprovechamos que estos se marcan de manera sencilla con la etiqueta `<a>` y le pedimos a BeautifulSoup que nos busque los enlaces.

In [ ]:
host = "https://es.wikipedia.org/wiki/Categor%C3%ADa:Pa%C3%ADses_del_mar_Caribe"

salsa = urllib.request.urlopen(host).read()
sopa =  BeautifulSoup(salsa, 'html.parser')

for contenido in sopa.select('#mw-pages a'):
	print(contenido)

El problema con este código es que en el momento de pasarlo por nuestra función saldrá un error:

In [ ]:
for contenido in sopa.select('#mw-pages a'):
	ws_texto(contenido)

El problema es que nuestro código muestra los enlaces de la siguiente manera:

```html
<a href="/wiki/Antigua_y_Barbuda" title="Antigua y Barbuda">Antigua y Barbuda</a>

```

Pero si copias ese texto en un navegador no te conduce al sitio. 

Necesitamos que ese texto se vea así:


```html
"https://es.wikipedia.org/wiki/Antigua_y_Barbuda"

```

Para ello necesitamos hacer dos cosas:

1. Vamos a separar el enlace en "host" y "ruta"

In [ ]:
host = "https://es.wikipedia.org"
ruta = "/wiki/Categor%C3%ADa:Pa%C3%ADses_del_mar_Caribe"

Luego, creamos la URL uniendo las dos partes, para lo cual utilizaremos la función `format()`:

In [ ]:
url = "{}{}".format(host, ruta)

print(url)

Y rehacemos nuestra sopa:

In [ ]:
salsa = urllib.request.urlopen(url).read() # cambiamos el enlace de 'host' a 'url'
sopa =  BeautifulSoup(salsa, 'html.parser')

2. Vamos a seleccionar solamente la ruta del enlace que se encuentra en el atributo `href="/wiki/Antigua_y_Barbuda"` Y uniremos esta ruta con la dirección del 'host' para construir nuestras URL

In [ ]:
for contenido in sopa.select('#mw-pages a'):
	obtener = "{}{}".format(host, contenido['href'])
	print(obtener)

Como verás, ahora el listado consiste en enlaces bien definidos. De hecho, puedes hacer click sobre cualquiera de ellos y verás que abre la página sin problemas. 
Esta es otra de las ventajas de este método. Como no haces un listado manual, si Wikipedia elimina o agrega nuevas páginas podrás acceder a los enlaces sin tener que lidiar con enlaces rotos.

El tercer paso consistirá en pasar nuestra función para cada enlace, con lo cual obtendremos el texto de cada página:

In [ ]:
for contenido in sopa.select('#mw-pages a'):
	obtener = "{}{}".format(host, contenido['href'])
	ws_texto(obtener)

Pero imprimir el texto en la pantalla no es de mucha utilidad, lo que querríamos sería tener el texto para luego analizarlo con otra librería o con otro software. Lo que vamos a hacer entonces será crear un directorio para guardar nuestros resultados y un archivo de texto para cada página:

In [ ]:
import os

path = os.getcwd()

directorio = '{}/descargas/'.format(path)

if not os.path.exists(directorio):
	os.makedirs(directorio)

os.listdir()

Modificamos la función para que en lugar de imprimir cree un archivo de texto. Vamos a aprovechar el módulo `codecs` para garantizar que el resultado esté codificado adecuadamente en `utf-8` y no tengamos problemas con las tildes y diéresis.

Para crear un archivo necesitamos utilizar la función `codecs.open('nombredelarchivo.txt')`, además, necesitamos agregar los parámetros `"a"`, para que cada título y párrafo se añadan al archivo de texto, y `"utf-8"` para evitar errores con la ortografía española.

Para identificar cada archivo, creamos seleccionamos el título del artículo y lo añadimos como nombre del archivo.

Además, para ayudar con la identificación del archivo vamos a agregar entre corchetes el idioma de la página. Así, el nombre del archivo quedará compuesto de tres componentes:

- La ruta (directorio)
- El lenguaje
- El título

Así, el artículo de, por ejemplo, Antigua y Barbuda, se llamará: `/directorio/[es]Antigua y Barbuda.txt` 

Para escribir el contenido utilizamos la función `write`, de tal manera que simplemente reemplazamos `print()` por `archiv.write()`.

Añadimos un condicional para evitar que descarguemos dos veces la misma información `if not os.path.exists(archivo):`

Nuestra función ahora se ve así:

In [ ]:
import codecs

def ws_texto(miurl, directorio): # Nuestra función requiere dos parámetros: una url (`miurl`) y un directorio
	salsa2 = urllib.request.urlopen(miurl).read()
	sopa2 =  BeautifulSoup(salsa2, 'html.parser')
	nombre = sopa2.find('h1', class_="firstHeading") # Título del artículo
	titulo = nombre.text
	lengua = extract = miurl.split('.')[-3].replace("https://", "") # Manipulación de la ruta para extraer el indicador de leng.
	archivo = "{}/[{}]{}.txt".format(directorio,lengua,titulo) # Construcción del nombre del archivo.
	if not os.path.exists(archivo): # Condicional -> Si no existe un archivo con ese nombre, escríbelo.
		archiv = codecs.open(archivo, "a", "utf-8")
		for titulo in sopa2.find_all(['h1','h2','h3']):
			archiv.write(titulo.text)
			for parrafo in titulo.next_siblings:
				if parrafo.name and parrafo.name.startswith('h'):
					break
				if parrafo.name == 'p':
					archiv.write(parrafo.text)
	else:
		print("Ya existe el archivo") # Mensaje si existe el archivo

Y ahora podemos correr nuestro script para obtener los archivos:

In [ ]:
for contenido in sopa.select('#mw-pages a'):
	obtener = "{}{}".format(host, contenido['href'])
	ws_texto(obtener, directorio)

En Google Colab podrás acceder a los archivos desde el menu de la columna izquierda, en la pestaña 'Archivos'

![](https://drive.google.com/uc?id=164YRYz1XiP0kyTCh9NR00vkN0lNoYGh5)

## Recuperar el texto de un artículo en todos los idiomas que tenga disponible

Para recuperar esta información tendremos que repetir algunos pasos. Primero, vamos a identificar la etiqueta que contiene los enlaces del menú `En otros idiomas`, para lo cual utilizaremos nuevamente la herramienta de inspección:

![](https://drive.google.com/uc?id=19BV56VAqQrvr3TxX48I0cgvJ1GtV8V7c)

Como verás, cada enlace corresponde a una etiqueta `a` con el atributo `class="interlanguage-link-target"`. Podemos observar también que el atributo `href` contiene un enlace completo, así que no será necesario "dividir" la url. 

Aprovecharemos nuestra función `ws_texto()` y solamente formaremos la ruta del directorio y de las url para recuperar la información.

In [ ]:
url = "https://es.wikipedia.org/wiki/Colombia" # Guardamos en una variable la url

salsa = urllib.request.urlopen(url).read()
sopa =  BeautifulSoup(salsa, 'html.parser') # Hacemos la sopa

path = os.getcwd()
directorio = '{}/interlingua/'.format(path) # Cambiamos un poco la ruta del directorio para distinguirla del ejercicio anterior

if not os.path.exists(directorio): # Creamos el directorio en caso de que no exista
	os.makedirs(directorio)
    
os.listdir()

Ahora, tendremos que recuperar primero la información de nuestra página `https://es.wikipedia.org/wiki/Colombia` para lo cual simplemente pasamos la url y la ruta del directorio por nuestra función `ws_texto()`

In [ ]:
ws_texto(url, directorio)

Posteriormente, creamos un loop en el que seleccionamos la información de los enlaces a las páginas interlengua `'a', class_="interlanguage-link-target"` y obtenemos el contenido del atributo `'href'`.

Finalmente, pasamos cada enlace por nuestra función `ws_texto()`.

In [ ]:
for contenido in sopa.find_all('a', class_="interlanguage-link-target"):
	obtener = contenido['href']
	ws_texto(obtener, directorio)

## Recuperar información y tabularla

Ahora vamos a hacer un ejercicio en el que recuperaremos la información de la tabla una página y la guardaremos en forma tabulada, es decir, en `csv`. Esta es una de las formas más comunes de guardar información que luego procederá a ser visualizada para su análisis. 

La forma más sencilla para manejar información tabular con Python es a través de la librería `pandas`.


In [ ]:
import pandas as pd

url =  'https://es.wikipedia.org/wiki/Anexo:Divisi%C3%B3n_pol%C3%ADtica_de_%C3%81frica'

tabla = pd.read_html(url)[1]
print(tabla)


Podemos escoger ciertas columnas, por ejemplo `País`, `Superficie` y `Población`:

In [ ]:
df = tabla[['País', 'Superficie (km²)', 'Población (est. 2016)[2]​']] 
# Observa que el indicador de la columna debe ser EXACTAMENTE IGUAL al encabezado

print(df)


Y podemos guardar la información de las tablas en un archivo `csv` para usarlo posteriormente:

In [ ]:
# Creamos un directorio para nuestro csv

path = os.getcwd()
directorio = '{}/csv/'.format(path) # Cambiamos un poco la ruta del directorio para distinguirla del ejercicio anterior

if not os.path.exists(directorio): # Creamos el directorio en caso de que no exista
	os.makedirs(directorio)
    
os.listdir()

# Guardamos nuestro archivo csv

df.to_csv('{}pandas.csv'.format(directorio, encoding="utf-8"))
# En caso de utilizar el archivo csv con MSExcel será necesario codificarlo como `encoding="utf-8-sig"`

## Tabular información no estructurada

Uno de los ejercicios más comunes consiste en recuperar información no estructurada y tabularla para poder realizar análisis de la información. Lastimosamente, `pandas` solamente nos regresa la información que es posible encontrar en tablas, pero no nos permite seleccionar los enlaces o hacer iteraciones entre los elementos.

Tratar con tablas es un poco más complejo que hacerlo con textos. Por ejemplo, si queremos recuperar los nombres de los países de las tablas disponibles en la página `Anexo:Países` de Wikipedia (https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses) vamos a tener que recuperar primero los enlaces y luego intentar descartar aquellos que no se refieran a nombres de países:

Si recuperamos los enlaces de la tabla de manera simple obtendremos el siguiente resultado:

In [ ]:
host = "https://es.wikipedia.org"
ruta = "/wiki/Anexo:Pa%C3%ADses" 
url = "{}{}".format(host,ruta)

salsa = urllib.request.urlopen(url).read()
sopa =  BeautifulSoup(salsa, 'html.parser')

for contenido in sopa.select('table a'): # Aquí seleccionamos todos los enlaces en una tabla
    obtener = "{}{}".format(host, contenido['href'])
    print(contenido.text)
    print(obtener)

Como verán, hay muchos resultados que no queremos: enlaces a imágenes, archivos, notas y sitios externos.
Para seleccionar solamente los enlaces que necesitamos vamos a utilizar la condición `if else` para saltar (`pass`) los resultados de esta manera:

Para las imágenes descartaremos aquellos enlaces que terminen en `.svg`
Para los demás elementos, descartaremos las rutas que inicien con enlaces a notas [#], o a archivos ["/wiki/Archivo:"] o a enlaces externos ["/w/"]

Finalmente, revisaremos que el resultado sea el que deseamos:

In [ ]:
host = "https://es.wikipedia.org"
ruta = "/wiki/Anexo:Pa%C3%ADses" 
url = "{}{}".format(host,ruta)

salsa = urllib.request.urlopen(url).read()
sopa =  BeautifulSoup(salsa, 'html.parser')

for contenido in sopa.select('table a'): # Aquí seleccionamos todos los enlaces en una tabla
    obtener = "{}{}".format(host, contenido['href']) # Como ya sabemos, esta línea nos recuperará los enlaces
    if obtener.endswith(".svg"):
        pass
    if contenido['href'].startswith("#") or contenido['href'].startswith("/wiki/Archivo:") or contenido['href'].startswith("/w/"):
        pass
    else: # Extraemos el texto que será el nombre del enlace y una url completa.
        print(contenido.text)
        print(obtener)


El resultado no es ideal, aún tenemos enlaces a páginas que no se refieren a países como "Presidencialismo" o "Parlamentarismo". Por lo pronto podemos pasar a resolver otros problemas y posteriormente nos encargaremos de estas páginas.

Vamos a hacer una tabla que contenga las siguientes columnas: "Nombres del país", "Población" y el "IDH"

En una página cualquiera de un país ubicamos la etiqueta con la cual extraer el nombre, que es bastante sencillo:

In [ ]:
url = "https://es.wikipedia.org/wiki/Afganist%C3%A1n"

salsa = urllib.request.urlopen(url).read()
sopa =  BeautifulSoup(salsa, 'html.parser')

titulo = sopa.select_one('h1', class_='firstHeading').text
print(titulo)

Encontrar el valor de la población es más complicado, debido a que no hay clases o identificadores en las celdas de las tablas de Wikipedia es necesario buscar por un texto clave. Para nuestro caso necesitamos que BeautifulSoup encuentre el texto en esta etiqueta:

```html
<td colspan="2" style="border:0;padding:1px 7px 1px 1px;">
35 688 822&nbsp;hab.&nbsp;(2019)</td>```

Después de probar diferentes claves, la única que dio resultado fue 'Censo'.

Al recuperar el texto ("35 688 822 hab. (2019)") requerimos que nos muestre únicamente los dígitos y nos elimine la fecha. Para ello tuvimos que aplicar un loop en el que se dividiera el texto y se seleccionaran solamente los números, además de reemplazar los corchetes, las comas y los espacios. El resultado es como sigue:

In [ ]:
celda = sopa.select_one('th:contains(Censo) + td').text
numers = [int(s) for s in celda if s.isdigit()]
stringnum = str(numers)
numero = stringnum.replace('[', '').replace(']','').replace(',','').replace(' ','')
censo = numero[:-4] # Con esta manipulación eliminamos los cuatro dígitos de la fecha

print(censo)

Teniendo la experiencia de la selección de la población podemos avanzar hacia la captura del valor del IDH, que es mucho más simple pues el valor numérico se puede obtener con una sencilla manipulación de la cadena de texto. Tuvimos que cambiar la coma por un punto en el separado para trabajar de manera más sencilla con el archivo csv.

In [ ]:
celda2 = sopa.select_one('th:contains(IDH) + td').text
idh = celda2[2:7].replace(',','.')

print(idh)

En conjunto, esto debería darnos el siguiente resultado en nuestra página de ejemplo:

In [ ]:
fila = "{},{},{}".format(titulo,censo,idh)

print(fila)


Como en los ejercicios anteriores, vamos a construir una función para facilitar la iteración sobre todos los elementos. 

In [ ]:
def wk_tabla(miurl):
    
    fila = []
    
    salsa = urllib.request.urlopen(miurl).read()
    sopa =  BeautifulSoup(salsa, 'html.parser')

    titulo = sopa.select_one('h1', class_='firstHeading').text
    
    celda = sopa.select_one('th:contains(Censo) + td').text
    numers = [int(s) for s in celda if s.isdigit()]
    stringnum = str(numers)
    numero = stringnum.replace('[', '').replace(']','').replace(',','').replace(' ','')
    censo = numero[:-4]

    celda2 = sopa.select_one('th:contains(IDH) + td').text
    idh = celda2[2:7].replace(',','.')
    
    fila.append(titulo)
    fila.append(censo)
    fila.append(idh)
    
    return fila

Ahora que tenemos nuestra función ya podemos iterar únicamente sobre los enlaces a países, pues solamente estos contienen la tabla de información geográfica desde la cual obtendremos la información de la población y del IDH. 
En este caso no utilizaremos un condicional, sino aplicaremos un estilo de Python: 'Easier to ask for forgiveness than permission'. Es decir, intentaremos (`try`) que nos recupere la información de nuestra función, si esto no sucede nos marcará una excepción (`AttributeError`), el cual dejaremos pasar para que continue el loop. De esta manera, solamente la información de los países será recuperada.

In [ ]:
host = "https://es.wikipedia.org"
ruta = "/wiki/Anexo:Pa%C3%ADses" 
url = "{}{}".format(host,ruta)

salsa = urllib.request.urlopen(url).read()
sopa =  BeautifulSoup(salsa, 'html.parser')

print(['País', 'Población', 'IDH'])

for contenido in sopa.select('table a'): # Aquí seleccionamos todos los enlaces en una tabla
    obtener = "{}{}".format(host, contenido['href']) # Como ya sabemos, esta línea nos recuperará los enlaces
    if obtener.endswith(".svg"):
        pass
    if contenido['href'].startswith("#") or contenido['href'].startswith("/wiki/Archivo:") or contenido['href'].startswith("/w/"):
        pass
    else:
        try:
            fila = wk_tabla(obtener)
            print(fila)
        except AttributeError:
            pass

Ahora, para guardar un archivo `csv` solamente debemos hacer unos pequeños cambios al código:

In [ ]:
import os
import codecs
import csv # Esta librería nos ayudará a crear un archivo csv.

path = os.getcwd()
directorio = '{}/csv/'.format(path) # Cambiamos un poco la ruta del directorio para distinguirla del ejercicio anterior

if not os.path.exists(directorio): # Creamos el directorio en caso de que no exista
    os.makedirs(directorio)

# Escribimos el encabezado de la tabla.
with codecs.open('{}csv_paises.csv'.format(directorio), encoding="utf-8", mode='a') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvfile.writerow(['País', 'Población', 'IDH'])
    
for contenido in sopa.select('table a'): # Aquí seleccionamos todos los enlaces en una tabla
    obtener = "{}{}".format(host, contenido['href']) # Como ya sabemos, esta línea nos recuperará los enlaces
    if obtener.endswith(".svg"):
        pass
    if contenido['href'].startswith("#") or contenido['href'].startswith("/wiki/Archivo:") or contenido['href'].startswith("/w/"):
        pass
    else: # Extraemos el texto que será el nombre del enlace y una url completa.
        try:
            fila = wk_tabla(obtener)
            # Escribimos cada fila de la tabla con los resultados que nos dará nuestra función.
            with codecs.open('{}csv_paises.csv'.format(directorio), encoding="utf-8", mode='a') as csvfile:
                csvfile = csv.writer(csvfile, delimiter=',')
                csvfile.writerow(fila)
                
        except AttributeError:
            pass


## Final

Con este ejercicio finalizamos nuestro taller. Muchas líneas de este código se elaboraron específicamente para este taller y pueden ser reutilizadas, modificadas y mejoradas. Espero que pueda ser de utilidad para sus investigaciones. 